# Devlog 2023-07-13

Now that we have done a compatibility pass with all of the system modules, let's do a compatibility test. The results of this test should comport with [the analysis in this spreadsheet](https://docs.google.com/spreadsheets/d/1FF_ovXYQUb65JagcLj8QCmCzs2_Pzd9qnZ5Cr28v4jw/edit?usp=sharing).

Generate the cross product of all available modules and test them one-by-one to see if they run successfully. We'll just run each simulation for a few days and verify that this doesn't throw any exceptions. Ideally, these results will line up with our expected compatibility matrix.

In [1]:
from datetime import date
import time
from epymorph.data import mm_library, geo_library, ipm_library
from itertools import product
import pandas as pd

from epymorph.simulation import Simulation

geo_order = ['pei', 'single_pop', 'us_counties_2015',
             'us_states_2015', 'maricopa_cbg_2019']
mm_order = ['pei', 'sparsemod', 'icecube', 'no']
ipm_order = ['pei', 'sirs', 'sirh', 'no']

# geo_order = ['pei', 'single_pop']
# mm_order = ['pei', 'sparsemod']
# ipm_order = ['pei', 'simple_sirs']

combos = pd.DataFrame(
    product(geo_order, mm_order, ipm_order),
    columns=['geo', 'mm', 'ipm']
)

# Params to satisfy every combo!
params = {
    'alpha': [0.05, 0.4125, 0.5875],
    'beta': 0.4,
    'gamma': 0.1000022,
    'xi': 1 / 90.0,
    'phi': 40,
    'theta': 0.1,
    'move_control': 0.9,
    'infection_duration': 4.0,
    'immunity_duration': 90.0,
    'hospitalization_duration': 14.0,
    'hospitalization_rate': 0.1,
    'infection_seed_loc': 0,
    'infection_seed_size': 100
}


def test(geo, mm, ipm):
    success = True
    t0 = time.perf_counter()
    try:
        Simulation(
            geo=geo_library[geo](),
            mvm_builder=mm_library[mm](),
            ipm_builder=ipm_library[ipm]()
        ).run(params, date(2010, 1, 1), 3)
    except:
        success = False
    t1 = time.perf_counter()
    print(f"{'succeeded' if success else 'failed'}: {geo, mm, ipm} in {((t1 - t0) * 1000.0):.3f} ms")
    return pd.Series([success, t1 - t0])


compat = combos.copy()
compat[['runs', 'runtime']] = combos.apply(
    lambda x: test(x['geo'], x['mm'], x['ipm']),
    axis=1)


succeeded: ('pei', 'pei', 'pei') in 52.424 ms
succeeded: ('pei', 'pei', 'sirs') in 19.034 ms
succeeded: ('pei', 'pei', 'sirh') in 121.063 ms
succeeded: ('pei', 'pei', 'no') in 11.470 ms
succeeded: ('pei', 'sparsemod', 'pei') in 16.382 ms
succeeded: ('pei', 'sparsemod', 'sirs') in 14.896 ms
succeeded: ('pei', 'sparsemod', 'sirh') in 23.327 ms
succeeded: ('pei', 'sparsemod', 'no') in 8.979 ms
succeeded: ('pei', 'icecube', 'pei') in 13.647 ms
succeeded: ('pei', 'icecube', 'sirs') in 13.227 ms
succeeded: ('pei', 'icecube', 'sirh') in 21.976 ms
succeeded: ('pei', 'icecube', 'no') in 4.031 ms
succeeded: ('pei', 'no', 'pei') in 9.654 ms
succeeded: ('pei', 'no', 'sirs') in 7.642 ms
succeeded: ('pei', 'no', 'sirh') in 15.160 ms
succeeded: ('pei', 'no', 'no') in 2.518 ms
failed: ('single_pop', 'pei', 'pei') in 6.286 ms
succeeded: ('single_pop', 'pei', 'sirs') in 9.092 ms
succeeded: ('single_pop', 'pei', 'sirh') in 17.210 ms
succeeded: ('single_pop', 'pei', 'no') in 5.734 ms
failed: ('single_pop'

In [4]:
from functools import partial, reduce

boolean_or = partial(reduce, bool.__or__)


mm_compat = compat.groupby(by=['geo', 'mm'])\
    .agg({'runs': boolean_or})\
    .reset_index()\
    .pivot(index='geo', columns='mm', values='runs')\
    .reset_index()\
    .reindex(columns=['geo', *mm_order])\
    .sort_values(by='geo', key=lambda x: x.map({k: v for v, k in enumerate(geo_order)}))

display(mm_compat)

ipm_compat = compat.groupby(by=['geo', 'ipm'])\
    .agg({'runs': boolean_or})\
    .reset_index()\
    .pivot(index='geo', columns='ipm', values='runs')\
    .reset_index()\
    .reindex(columns=['geo', *ipm_order])\
    .sort_values(by='geo', key=lambda x: x.map({k: v for v, k in enumerate(geo_order)}))

display(ipm_compat)

# NOTE: and when I ran this, it matched my expectations as laid out in this spreadsheet:
# https://docs.google.com/spreadsheets/d/1FF_ovXYQUb65JagcLj8QCmCzs2_Pzd9qnZ5Cr28v4jw/edit?usp=sharing


mm,geo,pei,sparsemod,icecube,no
1,pei,True,True,True,True
2,single_pop,True,True,True,True
3,us_counties_2015,True,True,True,True
4,us_states_2015,True,True,True,True
0,maricopa_cbg_2019,False,False,True,True


ipm,geo,pei,sirs,sirh,no
1,pei,True,True,True,True
2,single_pop,False,True,True,True
3,us_counties_2015,False,True,True,True
4,us_states_2015,False,True,True,True
0,maricopa_cbg_2019,False,True,True,True


In [5]:
# What are the slowest sims (that succeed)?
compat\
    .loc[compat['runs'] == True]\
    .sort_values(by='runtime', ascending=False)\
    .reset_index(drop=True)


,geo,mm,ipm,runs,runtime
0,us_counties_2015,sparsemod,sirh,True,79.086907
1,us_counties_2015,sparsemod,sirs,True,78.855257
2,us_counties_2015,sparsemod,no,True,73.821996
3,us_counties_2015,pei,sirh,True,25.175448
4,us_counties_2015,pei,sirs,True,24.253685
5,us_counties_2015,pei,no,True,22.880190
6,us_counties_2015,icecube,sirh,True,5.032540
7,us_counties_2015,icecube,sirs,True,4.552057
8,us_counties_2015,no,sirh,True,4.131392
9,us_counties_2015,no,sirs,True,3.963863
